<a href="https://colab.research.google.com/github/tirals88/DL_for_everyone/blob/main/deeplearning/notebook/17%EC%9E%A5%20RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

'모두의 딥러닝' 책 스터디 내용을 jupyter notebook으로 정리하여 올립니다.

Github 주소 : 'https://github.com/gilbutITbook/080228'

**모두의 딥러닝**

# 17 장 순환 신경망 RNN

문장의 학습 : 이전까지 학습했던 내용들과는 다르게 문장은 여러 개의 단어가 **정해진 순서대로 입력**되어야 한다.</br>다시 말해, 과거에 입력된 데이터와 나중에 입력된 데이터 사이의 관계를 고려해야 하는 문제가 생긴다.

이러한 유기성을 해결하기 위해 **순환 싱경망(Recurrent Neural Network, RNN)** 방법이 고안되었다.

순환 신경망은 여러 개의 데이터가 순서대로 입력되었을 때, 앞서 입력받은 데이터를 잠시 기억해 놓는 방법이다.

그리고 기억된 데이터가 얼마나 중요한지를 판단하여 별도의 가중치를 줘서 다음 데이터로 넘어간다.

## Long Short Term Memory (LSTM)

RNN이 처음 개발된 이후, RNN 결과를 개선하기 위해 발전된 방식들 중 LSTM이 널리 사용되었다.

이는 한 층 안에서 반복을 많이 해야 하는 RNN 특성상 일반 신경망보다 기울기 소실 문제가 더 많이 발생하고, 이를 해결하기 어렵다는 단점을 보완한 방법이다.

즉, 반복되기 직전 다음 층으로 기억된 값을 넘길지 안 넘길지를 관리하는 단계를 하나 더 추가하는 것이다.


RNN 방식의 장점은 입력 값과 출력 값을 어떻게 설정하느냐에 따라 여러 가지 상황에서 적용할 수 있다.

1) 다수 입력 / 단일 출력 : 문장을 읽고 뜻을 파악

2) 단일 입력 / 다수 출력 : 사진 캡션 생성

3) 다수 입력 / 다수 출력 : 문장 번역

## LSTM 을 이용한 로이터 뉴스 카테고리 분류하기

keras에서 load_data()를 통하여 MNIST 데이터셋과 같이 RNN 학습에 적절한 텍스트 대용량 데이터 또한 받을 수 있다.

케라스가 제공하는 '로이터 뉴스 카테고리 분류'와 'IMDB 영화 리뷰'를 통해 RNN을 학습해 볼 것이다.

입력된 문장의 의미를 파악하는 것은 곧 모든 단어를 종합하여 하나의 카테고리로 분류하는 작업이라고 할 수 있다.

예를 들어, "안녕. 오늘 날씨가 참 좋네"라는 말은 '인사'카테고리에 분류해야 한다.

이러한 긴 텍스트를 읽고 이 데이터가 어떤 의미를 지니는지를 카테고리로 분류하는 연습을 할 것이다.

**로이터 뉴스 데이터는, 총 11,258개의 뉴스 기사가 46개의 카테고리로 나누어진 대용량 텍스트 데이터이다.**

In [6]:
#로이터 뉴스 데이터셋 불러오기
import numpy as np
from keras.datasets import reuters

#데이터 분리

(X_train, Y_train), (X_test, Y_test) = reuters.load_data(num_words=1000, test_split=.2)

#데이터 확인
category = np.max(Y_train) + 1
print(category, '카테고리')
print(len(X_train), '학습용 뉴스 기사')
print(len(X_test), '테스트용 뉴스 기사')
print(X_train[0])

46 카테고리
8982 학습용 뉴스 기사
2246 테스트용 뉴스 기사
[1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 2, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 2, 2, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 2, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]


In [45]:
print(X_train[:20])
print(Y_train[:30])

[list([1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 2, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 2, 2, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 2, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12])
 list([1, 2, 699, 2, 2, 56, 2, 2, 9, 56, 2, 2, 81, 5, 2, 57, 366, 737, 132, 20, 2, 7, 2, 49, 2, 2, 2, 2, 699, 2, 8, 7, 10, 241, 16, 855, 129, 231, 783, 5, 4, 587, 2, 2, 2, 775, 7, 48, 34, 191, 44, 35, 2, 505, 17, 12])
 list([1, 53, 12, 284, 15, 14, 272, 26, 53, 959, 32, 818, 15, 14, 272, 26, 39, 684, 70, 11, 14, 12, 2, 18, 180, 183, 187, 70, 11, 14, 102, 32, 11, 29, 53, 44, 704, 15, 14, 19, 758, 15, 53, 959, 47, 2, 15, 14, 19, 132, 15, 39, 965, 32, 11, 14, 147, 72, 11, 180, 183, 187, 44, 11, 14, 102, 19, 11, 123, 186, 90, 67, 960, 4, 78, 13, 68, 467, 511, 110, 59, 89, 90, 67, 2, 55, 2, 92, 617, 80, 2, 46, 905, 220, 13, 4, 346, 48, 235, 62

In [40]:
#히스토그램이 균등하게 나오지 않는 것이 확인된다.
x = np.arange(0, 46, 1)
hist1, bins = np.histogram(weight1, x)
hist2, bins = np.histogram(weight2, x)

print(hist1)
print(hist2)

[  55  432   74 3159 1949   17   48   16  139  101  124  390   49  172
   26   20  444   39   66  549  269  100   15   41   62   92   24   15
   48   19   45   39   32   11   50   10   49   19   19   24   36   30
   13   21   30]
[ 12 105  20 813 474   5  14   3  38  25  30  83  13  37   2   9  99  12
  20 133  70  27   7  12  19  31   8   4  10   4  12  13  10   5   7   6
  11   2   3   5  10   8   3   6   6]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45]


그리고 학습 데이터를 보면 숫자 배열인 것이 확인 되는데 이 숫자는 데이터 안에서 해당 단어가 몇 번이나 나타나는지 세어 빈도에 따라 번호를 붙인 것이다.

1, 2, 2, 8, ... 의 1은 아마 'A', 'The' 즘 되지 않을까 싶다. 그렇지만 2가 두 번 연속 나오는 것에 대해서는 중복된 단어라기 보다는 빈도수가 같은 단어라고 생각이 든다.